# HSML - Iris flower classifier

In [1]:
import hsml

from hsml.kafka_topic_config import KafkaTopicConfig
from hsml.inference_logger_config import InferenceLoggerConfig
from hsml.resources_config import PredictorResourcesConfig
from hsml.predictor_config import PredictorConfig
from hsml.transformer_config import TransformerConfig

In [2]:
# connection
conn = hsml.connection()

# model registry
mr = conn.get_model_registry()

# model serving
ms = conn.get_model_serving()

Connected. Call `.close()` to terminate connection gracefully.


## Get best model

In [3]:
MODEL_NAME = "irisflowerclassifier"

In [4]:
# get best model
iris_model = mr.get_best_model(MODEL_NAME, "accuracy", "max")

## Deploy the model

### Option A. Using model _.deploy()_

In [5]:
# simplest syntax

iris_model.deploy() # model name -> deployment name

# or, 
iris_model.deploy(name="irisdeployment")

# or,
#iris_model.deploy(artifact_version=3) 

# ...

In [6]:
# with predictor config

PREDICTOR_SCRIPT = "/Projects/demo_ml_meb10000/Jupyter/serving/docker_and_kubernetes/python/predictor.py"

# resources config:  Optional
resources_config = PredictorResourcesConfig(num_instances=1,
                                   cores=1,
                                   memory=1024,
                                   gpus=0)
resources_config.describe()

# predictor config: 
predictor_config = PredictorConfig(model_server="PYTHON", # the only mandatory
                                   serving_tool="DEFAULT",
                                   script_file=PREDICTOR_SCRIPT,
                                   resources_config=resources_config,
                                   inference_logger=None,
                                   inference_batcher=None)
predictor_config.describe()

{
    "predictor_resources_config": {
        "cores": 1,
        "gpus": 0,
        "memory": 1024
    },
    "requested_instances": 1
}
{
    "batching_enabled": false,
    "inference_logging": "NONE",
    "model_server": "PYTHON",
    "predictor": "/Projects/demo_ml_meb10000/Jupyter/serving/docker_and_kubernetes/python/predictor.py",
    "predictor_resources_config": {
        "cores": 1,
        "gpus": 0,
        "memory": 1024
    },
    "requested_instances": 1,
    "serving_tool": "DEFAULT"
}


In [7]:
deployment = iris_model.deploy(name="irisclassifierfrommodel", predictor_config=predictor_config)
deployment.describe()

{
    "artifact_version": 5,
    "batching_enabled": false,
    "created": "2022-02-02T14:49:04.15Z",
    "creator": "Admin Admin",
    "id": 15,
    "inference_logging": "NONE",
    "model_name": "irisflowerclassifier",
    "model_path": "/Projects/demo_ml_meb10000/Models/irisflowerclassifier",
    "model_server": "PYTHON",
    "model_version": 1,
    "name": "irisclassifierfrommodel",
    "predictor": "predictor.py",
    "predictor_resources_config": {
        "cores": 1,
        "gpus": 0,
        "memory": 1024
    },
    "requested_instances": 1,
    "serving_tool": "DEFAULT"
}


### Option B. Creating a predictor and _.deploy()_

In [8]:
iris_predictor = ms.create_predictor(name="irisclassifierfrompredictor",
                                     model=iris_model,
                                     predictor_config=predictor_config)
deployment = iris_predictor.deploy()
deployment.to_dict()

{'id': 16,
 'name': 'irisclassifierfrompredictor',
 'modelName': 'irisflowerclassifier',
 'modelPath': '/Projects/demo_ml_meb10000/Models/irisflowerclassifier',
 'modelVersion': 1,
 'artifactVersion': 6,
 'created': '2022-02-02T14:49:14.998Z',
 'creator': 'Admin Admin',
 'modelServer': 'PYTHON',
 'servingTool': 'DEFAULT',
 'predictor': 'predictor.py',
 'requestedInstances': 1,
 'predictorResourcesConfig': {'cores': 1, 'memory': 1024, 'gpus': 0},
 'inferenceLogging': 'NONE',
 'batchingEnabled': False}

### Option C. Creating a deployment and _.save()_

In [9]:
iris_predictor = ms.create_predictor(model=iris_model,
                                     predictor_config=predictor_config)
deployment = ms.create_deployment(iris_predictor, name="irisclassifierfromdeployment")
deployment.save()

deployment.to_dict()

{'id': 17,
 'name': 'irisclassifierfromdeployment',
 'modelName': 'irisflowerclassifier',
 'modelPath': '/Projects/demo_ml_meb10000/Models/irisflowerclassifier',
 'modelVersion': 1,
 'artifactVersion': 7,
 'created': '2022-02-02T14:49:25.703Z',
 'creator': 'Admin Admin',
 'modelServer': 'PYTHON',
 'servingTool': 'DEFAULT',
 'predictor': 'predictor.py',
 'requestedInstances': 1,
 'predictorResourcesConfig': {'cores': 1, 'memory': 1024, 'gpus': 0},
 'inferenceLogging': 'NONE',
 'batchingEnabled': False}

### More complex deployment

In [14]:
PREDICTOR_SCRIPT = "/Projects/demo_ml_meb10000/Jupyter/serving/kfserving/python/predictor.py"
TRANSFORMER_SCRIPT = "/Projects/demo_ml_meb10000/Jupyter/serving/kfserving/sklearn/transformer.py"

predictor_config = PredictorConfig(model_server="PYTHON",
                                   serving_tool="KFSERVING",
                                   script_file=PREDICTOR_SCRIPT)

transformer_config = TransformerConfig(script_file=TRANSFORMER_SCRIPT)

iris_predictor = ms.create_predictor(iris_model,
                                     name="iriscomplexclassifier",
                                     predictor_config=predictor_config,
                                     transformer_config=transformer_config)

iris_deployment = iris_predictor.deploy()

## Get deployments

In [15]:
# get all
[d.name + " : " + str(d.id) for d in ms.get_deployments()]

['irisdeployment : 14',
 'irisflowerclassifier : 13',
 'irisclassifierfromdeployment : 17',
 'irisclassifierfrommodel : 15',
 'irisclassifierfrompredictor : 16',
 'iriscomplexclassifier : 18']

## Operations on deployments

In [16]:
# describe
iris_deployment = ms.get_deployment("iriscomplexclassifier")
iris_deployment.describe()

{
    "artifact_version": 8,
    "batching_enabled": false,
    "created": "2022-02-02T14:52:00Z",
    "creator": "Admin Admin",
    "id": 18,
    "inference_logging": "NONE",
    "model_name": "irisflowerclassifier",
    "model_path": "/Projects/demo_ml_meb10000/Models/irisflowerclassifier",
    "model_server": "PYTHON",
    "model_version": 1,
    "name": "iriscomplexclassifier",
    "predictor": "predictor.py",
    "predictor_resources_config": {
        "cores": 1,
        "gpus": 0,
        "memory": 1024
    },
    "requested_instances": 1,
    "requested_transformer_instances": 1,
    "serving_tool": "KFSERVING",
    "transformer": "transformer.py",
    "transformer_resources_config": {
        "cores": 1,
        "gpus": 0,
        "memory": 1024
    }
}


In [17]:
# get state
deployment_state = iris_deployment.get_state()
deployment_state.describe()

{
    "available_instances": 0,
    "available_transformer_instances": 0,
    "deployed": false,
    "internal_ips": [
        "10.0.2.15"
    ],
    "internal_path": "/v1/models/iriscomplexclassifier",
    "internal_port": 30824,
    "status": "Stopped"
}


In [18]:
# start
iris_deployment.start()

Deployment has not reached the desired status within the expected awaiting time, set a higher value for await_running to wait longer.


In [ ]:
# get state
deployment_state = iris_deployment.get_state()
deployment_state.describe()

In [ ]:
# stop
iris_deployment.stop()

In [ ]:
# get state
deployment_state = deployment.get_state()
deployment_state.describe()

## More complex use

In [ ]:
PREDICTOR_SCRIPT = "/Projects/demo_ml_meb10000/Jupyter/serving/kfserving/python/predictor.py"
TRANSFORMER_SCRIPT = "/Projects/demo_ml_meb10000/Jupyter/serving/kfserving/sklearn/transformer.py"

predictor_config = PredictorConfig(model_server="PYTHON",
                                   serving_tool="KFSERVING",
                                   script_file=PREDICTOR_SCRIPT)

transformer_config = TransformerConfig(script_file=TRANSFORMER_SCRIPT)

iris_predictor = ms.create_predictor(iris_model,
                                     predictor_config=predictor_config,
                                     transformer_config=transformer_config)

In [ ]:
iris_deployment = iris_predictor.deploy()

In [ ]:
iris_deployment.start()

In [ ]:
MODEL_NAME = "irisflowerclassifier"

# get all
[d.name + " : " + str(d.id) for d in ms.get_deployments()]

In [ ]:
# get by id
iris_deployment = ms.get_deployment(1061)
iris_deployment.describe()

In [ ]:
iris_deployment.start()

In [ ]:
iris_deployment.get_state().status

## Make inference  (PENDING)

In [ ]:
import json
import random

NUM_FEATURES = 4

data = {"inputs" : [[random.uniform(1, 8) for i in range(NUM_FEATURES)]]}
response = iris_deployment.predict(data)
print(response)

## Delete deployment

In [ ]:
deployment.delete()